### RDKit Conformer Browser
Derived from Greg Landrum's code by Malitha Humayun Kabir<br>
As a part of GSoC 2017 <br>
Project : RDKit - 3Dmol.js integration <br>
Mentors: Paul Czodrowski and Greg Landrum <br>
Date: 28th August 2017 <br>
Acknowledgement: Peter Gedeck reviewed source codes and provided advice on restructuring codes and wrote MolViewState class.

##### This notebook assumes that  IPythonConsoleIntegration and LigandExtract merged in RDKit code base. 

In [1]:
from rdkit.Chem.Draw import IPythonConsoleIntegration as ipy
from rdkit import Chem
from rdkit.Chem import AllChem
from six.moves import urllib

If you we use a new notebook, first task is to load the 3Dmol.js libaray. The following cell deos this

In [2]:
%%html
<script href="https://raw.githubusercontent.com/3dmol/3Dmol.js/master/release/3Dmol.js"></script>

Now we need to have molecules and different conformers of molecule. The following cell creates a list of molecule where each molecule got 10 conformers.

In [3]:
# Codes from Paolo Tosco (Slightly modified version)
url = 'https://github.com/rdkit/rdkit/raw/master/Docs/Book/data/cdk2.sdf'
response = urllib.request.urlopen(url)
data = response.read()
suppl = Chem.SDMolSupplier()
suppl.SetData(data)
molList = [mol for mol in suppl]
params = AllChem.ETKDG()
params.numThreads=3
for m in molList:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)


In [4]:
print(len(molList))

47


Great! We now have a list of molecules. But MolView3D doesn't support list object rather it supports the dictionary. As you aleady know that a dictionary must have keys. So, we need to add keys in the molecule list first. For the convenience of demonstration, we will be using only 4 molecules from the list and then will convert the list to dictionary.

In [5]:
molList_2 = molList[:4]

In [6]:
molList_2_with_expected_keys=[(str(ids),mol) for ids,mol in enumerate(molList_2)]

In [7]:
molList_2_with_expected_keys

[('0', <rdkit.Chem.rdchem.Mol at 0x7febe132e1a0>),
 ('1', <rdkit.Chem.rdchem.Mol at 0x7febe132e210>),
 ('2', <rdkit.Chem.rdchem.Mol at 0x7febe132e280>),
 ('3', <rdkit.Chem.rdchem.Mol at 0x7febe132e2f0>)]

Now we can convert this list to dictionary. Please see the following cell.

In [8]:
molDict = dict(molList_2_with_expected_keys)

In [9]:
molDict

{'0': <rdkit.Chem.rdchem.Mol at 0x7febe132e1a0>,
 '1': <rdkit.Chem.rdchem.Mol at 0x7febe132e210>,
 '2': <rdkit.Chem.rdchem.Mol at 0x7febe132e280>,
 '3': <rdkit.Chem.rdchem.Mol at 0x7febe132e2f0>}

The keys of the molDict keys will be considered as molecule id (molId). Please note that: MolView3D currently does NOT accept molId other than str type.

In [10]:
print('molecule keys within dictionary is : ' + str(type(list(molDict.keys())[0])))
print('The following are the molecule keys : ')
print(list(molDict.keys()))

molecule keys within dictionary is : <type 'str'>
The following are the molecule keys : 
['1', '0', '3', '2']


In [11]:
moldictForVisualization = ipy.ProcessLigandDict(molDict)

In [12]:
view = ipy.MolView3D(ligandDict = moldictForVisualization, 
                     ligSelPanel = 'minimal', 
                     ligStyle = 'line')

If you want to retrieve the Ids or selected molecule and conformers then run the following codes.

In [13]:
view.molViewState.idPaired

(('1', 0),)

The retuned value of view.molViewState.idPaired is either list containing tuple. The tuple inside the list is paired value where the first value is molecule id (the keys of unprocessed dictionary) and the second value is the conformer id of the selected molecule. You can use this syntax to select conformers using python codes. But in this case only tuple conatining tuple is acceptable. 

In [14]:
view.SelectMolAndConf(molAndConfIds = (('0',0),))

You can also select multiple conformer by using the conformer selection syntax.

In [15]:
view.SelectMolAndConf(molAndConfIds = (('3',7),('1',0),))

As you may notice that the molecule containing SD file contained few properties for  each molecule. You can view those by enabling the propertyPanel. 

In [16]:
view_2 = ipy.MolView3D(ligandDict = moldictForVisualization, 
                     ligSelPanel = 'minimal', 
                     ligStyle = 'line',
                     propertyPanel = True)

Please not that propertyPanel will not work in case of multiple molecule selection.

You can see the conformer label and atom label in the molecule viewer too by parsing another argument labelPanel = True. Please note that this panel will only work for single conformer.

In [17]:
view_3 = ipy.MolView3D(ligandDict = moldictForVisualization, 
                       ligSelPanel = 'minimal', 
                       ligStyle = 'line',
                       propertyPanel = True,
                       labelPanel = True)

If you want to compare all the conformer geometry visually then you may need to use ligSelPanel = 'full'.

In [18]:
view_4 = ipy.MolView3D(ligandDict = moldictForVisualization, 
                       ligSelPanel = 'full', 
                       ligStyle = 'line',
                       propertyPanel = True,
                       labelPanel = True)

We also can also change molecule representation style. To do this we can use stylePanel = 'lig'. 

In [19]:
view_5 = ipy.MolView3D(ligandDict = moldictForVisualization, 
                       ligSelPanel = 'full', 
                       ligStyle = 'line',
                       propertyPanel = True,
                       labelPanel = True,
                       stylePanel = 'lig')

Please note: due to some unknown reasons, molecule/conformer selection through python codes are not working for stick models in some browser. We are trying to solve this issue.

Anoter note: We still have not seen what is written in moldictForVisualization. Let's see it!

In [20]:
moldictForVisualization

{'0': {'parent': <rdkit.Chem.rdchem.Mol at 0x7febe132e1a0>},
 '1': {'parent': <rdkit.Chem.rdchem.Mol at 0x7febe132e210>},
 '2': {'parent': <rdkit.Chem.rdchem.Mol at 0x7febe132e280>},
 '3': {'parent': <rdkit.Chem.rdchem.Mol at 0x7febe132e2f0>}}

And our first created dict looked like:

In [21]:
molDict

{'0': <rdkit.Chem.rdchem.Mol at 0x7febe132e1a0>,
 '1': <rdkit.Chem.rdchem.Mol at 0x7febe132e210>,
 '2': <rdkit.Chem.rdchem.Mol at 0x7febe132e280>,
 '3': <rdkit.Chem.rdchem.Mol at 0x7febe132e2f0>}

So, the function ProcessLigandDict actually adds another key named 'parent'. We can add custom keys replacing the default key - 'parent'. We will discuss it in the notebook demostrating energy minimization.

### Thank you very much for your time. Have a great day! 